In [1]:
import os
os.chdir("/home/sunil/Desktop/LT-TextMining/day 2")
import pandas as pd

# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size )

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [4]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [6]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [7]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train["review"]:
    review = review.decode("utf8")
    sentences += review_to_sentences(review, tokenizer)
    
print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    review = review.decode("utf8")
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/home/sunil/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/sunil/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/sunil/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
/home/sunil/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL.

Parsing sentences from unlabeled set


/home/sunil/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/sunil/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/sunil/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like r

In [8]:
print len(sentences)

795538


In [9]:
print sentences[0]

[u'with', u'all', u'this', u'stuff', u'going', u'down', u'at', u'the', u'moment', u'with', u'mj', u'i', u've', u'started', u'listening', u'to', u'his', u'music', u'watching', u'the', u'odd', u'documentary', u'here', u'and', u'there', u'watched', u'the', u'wiz', u'and', u'watched', u'moonwalker', u'again']


In [11]:
print sentences[1]

[u'maybe', u'i', u'just', u'want', u'to', u'get', u'a', u'certain', u'insight', u'into', u'this', u'guy', u'who', u'i', u'thought', u'was', u'really', u'cool', u'in', u'the', u'eighties', u'just', u'to', u'maybe', u'make', u'up', u'my', u'mind', u'whether', u'he', u'is', u'guilty', u'or', u'innocent']


In [10]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

import os
os.chdir("/home/sunil/Desktop/LT-TextMining/day 2")
# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

Training model...


In [11]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [12]:
model.doesnt_match("france england germany berlin".split())

'berlin'

In [13]:
model.doesnt_match("paris berlin london austria".split())

'paris'

In [14]:
model.most_similar("man")

[(u'lady', 0.6035962104797363),
 (u'woman', 0.6007094383239746),
 (u'lad', 0.5859739780426025),
 (u'monk', 0.5632014870643616),
 (u'chap', 0.5372190475463867),
 (u'businessman', 0.5286189913749695),
 (u'men', 0.5154789090156555),
 (u'guy', 0.5131467580795288),
 (u'person', 0.5112289190292358),
 (u'doctor', 0.5103709101676941)]

In [15]:
model.most_similar("queen")

[(u'princess', 0.676252007484436),
 (u'belle', 0.6154158115386963),
 (u'stepmother', 0.6143475770950317),
 (u'maid', 0.6079977750778198),
 (u'goddess', 0.5993334054946899),
 (u'mistress', 0.5953872203826904),
 (u'eva', 0.5872282981872559),
 (u'bride', 0.5827389359474182),
 (u'showgirl', 0.5754068493843079),
 (u'starlet', 0.5710420608520508)]

In [16]:
model.most_similar("awful")

[(u'terrible', 0.7907817363739014),
 (u'horrible', 0.7432720065116882),
 (u'atrocious', 0.7376508712768555),
 (u'dreadful', 0.7288143634796143),
 (u'abysmal', 0.6914944648742676),
 (u'horrid', 0.6884881258010864),
 (u'horrendous', 0.6846942901611328),
 (u'appalling', 0.6843228340148926),
 (u'lousy', 0.6491025686264038),
 (u'amateurish', 0.6302037239074707)]

In [21]:
model.most_similar("enemy")

[(u'operation', 0.6688388586044312),
 (u'army', 0.638384997844696),
 (u'allies', 0.6298497915267944),
 (u'dictatorship', 0.6264623403549194),
 (u'organization', 0.6220119595527649),
 (u'troops', 0.6081861257553101),
 (u'allied', 0.607942521572113),
 (u'enemies', 0.5969444513320923),
 (u'alliance', 0.5944496393203735),
 (u'opponent', 0.5838886499404907)]

In [22]:
model.most_similar("friend")

[(u'girlfriend', 0.6950176954269409),
 (u'roommate', 0.6709270477294922),
 (u'pal', 0.6537691354751587),
 (u'friends', 0.649190366268158),
 (u'cousin', 0.6284669637680054),
 (u'boyfriend', 0.626656174659729),
 (u'neighbor', 0.6210215091705322),
 (u'nephew', 0.5829442739486694),
 (u'partner', 0.5806877613067627),
 (u'grandfather', 0.575136661529541)]

In [18]:
# How do we use the word2vec model?
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

In [19]:
type(model.syn0)

numpy.ndarray

In [20]:
model.syn0.shape

(16490, 300)

In [22]:
model["flower"]

array([-0.09417266, -0.0378619 ,  0.00238338,  0.0462037 , -0.03795527,
        0.1099241 ,  0.07731149,  0.07177036, -0.00266445, -0.04181629,
       -0.05879096, -0.062683  ,  0.10843989,  0.06629829,  0.00277652,
       -0.04893847, -0.02048971, -0.02935813, -0.10809319, -0.08091625,
       -0.11017673, -0.00890051, -0.00737948,  0.02945272, -0.05598565,
        0.04972837, -0.07007974, -0.10799815, -0.06435319,  0.05343212,
        0.00549324,  0.02602439, -0.0135174 , -0.05806872, -0.00356912,
        0.08385617, -0.01578583, -0.02133882,  0.08826329,  0.05052979,
       -0.03038925, -0.01578024, -0.0090985 , -0.03386888, -0.0055368 ,
       -0.06235744, -0.03187468, -0.0486663 , -0.02738824,  0.05039812,
       -0.03355173,  0.00794848,  0.10176383,  0.02476787, -0.06586023,
        0.03704588,  0.03012166, -0.04328536, -0.08953732,  0.00728413,
       -0.01749072,  0.03924608,  0.03775894, -0.03532573, -0.07392065,
        0.0279113 ,  0.01902047,  0.03920373,  0.04916391, -0.07

In [23]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
        if counter%1000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs

In [24]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print "Creating average feature vecs for test reviews"
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

In [27]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
import numpy as np
forest = RandomForestClassifier( n_estimators = 100 )

print "Fitting a random forest to labeled training data..."
forest = forest.fit( trainDataVecs, train["sentiment"] )
# random forest performance through cross vaidation
print forest
print np.mean(cross_val_score(forest,trainDataVecs,train["sentiment"],cv=10))

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
0.83876
